In [0]:
import pandas as pd
import numpy as np
import datetime as dt

Read in the data file as a csv file

In [0]:
data = pd.read_csv('moloco.csv')

In [105]:
data.head()

,ts,user_id,country_id,site_id
0,2019-02-01 00:01:24,LC36FC,TL6,N0OTG
1,2019-02-01 00:10:19,LC39B6,TL6,N0OTG
2,2019-02-01 00:21:50,LC3500,TL6,N0OTG
3,2019-02-01 00:22:50,LC374F,TL6,N0OTG
4,2019-02-01 00:23:44,LCC1C3,TL6,QGO3G


The three columns in the data accounts for the timestamp, the user_id, country_id and the site_id

# EXPLORATORY DATA ANALYSIS

In [106]:
data.info() #we have non-null values and and all columns are object data types

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3553 entries, 0 to 3552
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ts          3553 non-null   object
 1   user_id     3553 non-null   object
 2   country_id  3553 non-null   object
 3   site_id     3553 non-null   object
dtypes: object(4)
memory usage: 111.2+ KB


In [107]:
data.user_id.nunique() # there are 1916 unique users in the data

1916

# Consider only the rows with country_id = "BDV" (there are 844 such rows). For each site_id, we can compute the number of unique user_id's found in these 844 rows. Which site_id has the largest number of unique users? And what's the number?

In [108]:
data.country_id.unique() ## this method computes the unique country_ids in the data

array(['TL6', 'HVQ', 'QLT', 'BDV', 'XA7', 'NVV', 'K1R'], dtype=object)

In [0]:
new_data  = data[data['country_id'] == 'BDV']

In [110]:
new_data.groupby(['site_id'])['user_id'].nunique()

site_id
3POLC      2
5NPAU    544
N0OTG     90
Name: user_id, dtype: int64

**Answer **- site_id 5NPAU has 544 unique users

# Between 2019-02-03 00:00:00 and 2019-02-04 23:59:59, there are four users who visited a certain site more than 10 times. Find these four users & which sites they (each) visited more than 10 times. (Simply provides four triples in the form (user_id, site_id, number of visits) in the box below.)

In [0]:
data['ts'] = pd.to_datetime(data['ts'])

In [112]:
data.info() #timestamp column has been converted to datetime format

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3553 entries, 0 to 3552
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ts          3553 non-null   datetime64[ns]
 1   user_id     3553 non-null   object        
 2   country_id  3553 non-null   object        
 3   site_id     3553 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 111.2+ KB


In [0]:
time_data  = data[(data['ts'] >= '2019-02-03 00:00:00') & (data['ts'] <= '2019-02-04 23:59:59')]

In [114]:
time_data.groupby(['site_id', 'user_id'])['user_id'].count().sort_values().tail(4)

site_id  user_id
3POLC    LC3C7E     15
N0OTG    LC3C9D     17
         LC06C3     25
         LC3A59     26
Name: user_id, dtype: int64

# For each site, compute the unique number of users whose last visit (found in the original data set) was to that site. For instance, user "LC3561"'s last visit is to "N0OTG" based on timestamp data. Based on this measure, what are top three sites? (hint: site "3POLC" is ranked at 5th with 28 users whose last visit in the data set was to 3POLC; simply provide three pairs in the form (site_id, number of users).)


In [0]:
data_freq = data.groupby(['user_id'])['ts'].max().sort_values()

In [0]:
users  = data_freq.to_frame().reset_index()

In [0]:
joined  = pd.merge(data,users, on=['user_id', 'ts'], how = 'right')

In [118]:
joined.site_id.value_counts()

5NPAU    992
N0OTG    561
QGO3G    289
GVOFK     42
3POLC     28
RT9Z6      2
EUZ/Q      1
JSUUP      1
Name: site_id, dtype: int64

# For each user, determine the first site he/she visited and the last site he/she visited based on the timestamp data. Compute the number of users whose first/last visits are to the same website. What is the number?

In [0]:
first_site = data.groupby(['user_id'])['ts'].min().reset_index()

In [0]:
last_site = data.groupby(['user_id'])['ts'].max().reset_index()



In [0]:
#merge both dataframes to the original data
first_joined  = pd.merge(data,first_site, on=['user_id', 'ts'], how = 'right')
last_joined  = pd.merge(data,last_site, on=['user_id', 'ts'], how = 'right')

In [122]:
same_website  = pd.merge(first_joined,last_joined, how = 'inner')
same_website.user_id.nunique()

1261